# Tarefa 5 - Aprendizado de Máquina 2023.02
Aluno: Diego Vasconcelos Schardosim de Matos - 120098723

Esta tarefa épara ser reailizada com o dataset IRIS. 

0. Separar 80% dos dados para treinamento e 20% para teste. Esse conjunto de teste será usado apenas no final de todo o trabalho. O conjunto de treino e teste deve ser balanceado (mesmo número de instâncias de cada classe).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
df = pd.read_csv("./iris.csv")
df.head()

In [ ]:
df.shape

In [ ]:
Y = df['target']
X = df.drop('target', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
df['target'].value_counts().plot.pie(autopct='%3.1f%%', shadow=True, legend=True, startangle=45)
plt.title('Distribuição do atributo de saída')
plt.show()

O dataset está bem distribuído

1. Para os 80% restantes, você vai usar K-fold com K = 5, para várias arquiteturas da rede neural.

In [ ]:
kf = KFold(n_splits=5)

2. Inicialmente, utilize todas as variávéis de entrada do dataset.
3. Utilize o classificador multi perceptron MLPClassifier com os seguintes valores:
- função de ativação: ‘logistic’, ‘tanh’, ‘relu’
- solver: ‘sgd’
- os outros parâmetros devem ser os que são default.
- o número de camadas e neurônios por camada ficam a seu critério
4. Para cada fold:
- informe o score de treinamento e teste
- plote um gráfico com o comportamento da função de perda. Você pode plotar em um gráfico só, todas as funções de perda.

In [ ]:
# Antes de treinar o Modelo, devo buscar por outliers e normalizar os dados 

X_train.plot(kind="box", subplots=True, title="Outliers")

Ao rodar o algoritmo acima algumas vezes, percebi que dependendo da seleção das instancias de teste pode haver ou não outliers na feature sepalWidthInCm.

Como o dataset é pequeno, é possível que remover os outliers interfira na acurácia do treinamento, então irei testar o modelo com e sem os outliers.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [ ]:
def train_my_model(model: MLPClassifier, axs = None, withOutliers=True) -> list:
    acc_score = []

    iteration = 0
    for train_index, test_index in kf.split(X):
        kf_x_train, kf_x_test = X.iloc[train_index,:], X.iloc[test_index,:]
        kf_y_train, kf_y_test = Y[train_index], Y[test_index]

        # Remover outliers
        if withOutliers == False:
            Q1 = kf_x_train['sepalWidthInCM'].quantile(0.25)
            Q3 = kf_x_train['sepalWidthInCM'].quantile(0.75)
            IQR = Q3 - Q1

            outliers = kf_x_train.loc[(kf_x_train['sepalWidthInCM'] < Q1 - 1.5 * IQR) | (kf_x_train['sepalWidthInCM'] > Q3 + 1.5 * IQR)]
            kf_x_train = kf_x_train.drop(outliers.index)
            kf_y_train = kf_y_train.drop(outliers.index)
        
        # Normalizar
        # scaler = MinMaxScaler()
        # columns = ['sepalLengthInCM', 'sepalWidthInCM', 'petalLengthInCM', 'petalWidthInCM']
        # kf_x_train_normalized = kf_x_train.copy()
        # kf_x_train_normalized[columns] = scaler.fit_transform(kf_x_train[columns])

        # kf_x_test_normalized = kf_x_test.copy()
        # kf_x_test_normalized[columns] = scaler.fit_transform(kf_x_test[columns])

        model.fit(kf_x_train, kf_y_train)
        acc = model.score(kf_x_test, kf_y_test)
        acc_score.append(acc)

        if axs is not None:
            axs[iteration].plot(model.loss_curve_, label='Função de perda')
            axs[iteration].set_title(f'KFold {iteration}')
            iteration += 1
    
    return acc_score

In [ ]:
for activation_function in ['logistic', 'tanh', 'relu']:
    print('=== Treinando com função de ativação: {} ==='.format(activation_function))

    model = MLPClassifier(activation=activation_function, solver='sgd')

    fig, axs = plt.subplots(nrows=1, ncols=5, sharey=True)
    acc_score = train_my_model(model, axs)
    fig.suptitle(f"Função de perda para {activation_function}")

    avg_acc_score = sum(acc_score) / 5

    print('Resultados do treinamento')  
    print('Accuracy of each fold: {}'.format(acc_score))
    print('Avg accuracy: {}'.format(avg_acc_score))


In [ ]:
for activation_function in ['logistic', 'tanh', 'relu']:
    print('=== Treinando com função de ativação: {} ==='.format(activation_function))

    model = MLPClassifier(activation=activation_function, solver='sgd')

    fig, axs = plt.subplots(nrows=1, ncols=5, sharey=True)
    acc_score = train_my_model(model, axs, withOutliers=False)
    fig.suptitle(f"Função de perda para {activation_function}")

    avg_acc_score = sum(acc_score) / 5

    print('Resultados do treinamento')  
    print('Accuracy of each fold: {}'.format(acc_score))
    print('Avg accuracy: {}'.format(avg_acc_score))


5. Analise o que ocorreu. 

Usando a técnica de reamostragem do K-Fold Cross Validation e o modelo de treinamento de Redes Neurais com funções de ativação igual a Logistica, Tanh ou Relu no aprendizado dos pesos de cada neuronio (backpropagation), o modelo não conseguiu convergir para uma solução dentro do limite máximo de 200 iterações.

Também não houve diferença remnovendo ou mantendo os outliers

6. Ainda usando os 80% dos dados e K-fold=5, repita o experimento alterados os parâmetros anteriores. A escolha de qual(is) parâmetro(s) será(ão) alterados deve ser justificado. O que você espera alterando o(s) parâmetro(s)? O resultado obtido foi o esperado?

Me vem em mente alterar quatro parametros:
- Taxa de aprendizado: O valor da taxa de aprendizado padrão é muito baixo (0.001) fazendo com que demore para convergir, penso em deixa-lo 10x maior
- Limite de iterações: Aumentar o limite máximo de iterações do modelo
- Diminuir o numero de neuronios da rede para deixa-la mais simples. Pode ser que não precise de todos os 100 (valor padrão da biblioteca)
- De acordo com a documentação da biblioteca, usar o método solver='lbfgs' converge melhor a para dataset pequenos. Como nosso caso possui apenas 150 instancias, estarei testando com ele

In [ ]:
for activation_function in ['logistic', 'tanh', 'relu']:
    print('=== Treinando com função de ativação: {} ==='.format(activation_function))
    
    model = MLPClassifier(hidden_layer_sizes=(10,), learning_rate_init=0.01, max_iter=1000, activation=activation_function, solver='lbfgs')

    acc_score = train_my_model(model)

    avg_acc_score = sum(acc_score) / 5

    print('Resultados do treinamento')  
    print('Accuracy of each fold: {}'.format(acc_score))
    print('Avg accuracy: {}'.format(avg_acc_score))


In [ ]:
for activation_function in ['logistic', 'tanh', 'relu']:
    print('=== Treinando com função de ativação: {} ==='.format(activation_function))
    
    model = MLPClassifier(hidden_layer_sizes=(10,), learning_rate_init=0.01, max_iter=1000, activation=activation_function, solver='lbfgs')

    acc_score = train_my_model(model, withOutliers=False)

    avg_acc_score = sum(acc_score) / 5

    print('Resultados do treinamento')  
    print('Accuracy of each fold: {}'.format(acc_score))
    print('Avg accuracy: {}'.format(avg_acc_score))


Como esperado, alterando os parametros mencionados conseguimos uma convergencia rápida e uma boa acurácia.

7- Após terminar todos os experientos, escolha o modelo gerado que você considera o melhor resultado. Usando os parâmetros deste modelo, faça um novo treinamento usando todos os 80% dos dados.

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(10,), learning_rate_init=0.01, max_iter=1000, activation='tanh', solver='lbfgs')
model.fit(X_train, y_train)

8- Avalie o modelo gerado no item 8 usando as 15 instâncias que não foram usadas. 

In [ ]:
pred_values = model.predict(X_test)
acc = accuracy_score(pred_values, y_test)

print('Resultados do treinamento')
print('Acurácia: {}'.format(acc))